In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from langdetect import detect
import os
import re
import liste_muvra
from lxml import etree
import bs4 as bs
import glob
import shutil

### Listage des fichiers TXT et transformation en fichiers XML

In [2]:
parent = os.getcwd()

In [3]:
list_files = glob.glob("./text_section/*.txt")

In [4]:
for file in list_files:
    with open(file, 'r') as f:
        content = f.readlines()
    
    t = ''.join(content)
    
    t = re.sub(r"^ ", r"", t)
    
    t = re.sub(r"^\n", r"toto", t)
    t = re.sub(r"^[^\w]+", r"", t)
    t = re.sub(r"^toto\n", r"", t)
    
    t = re.sub(r"-\n", r"", t)
    t = re.sub(r"\n", r" ", t)
    t = re.sub(r"<section>", r"\n", t)
    t = re.sub(r"^section>", r"", t)
        
    with open(file, 'w') as f :
        f.write(t)

In [5]:
list_aut_str = liste_muvra.auteurs_muvra
list_regiolect = liste_muvra.regiolectes
for file in list_files:
    with open(file, "r") as f:
        text = f.readlines()
    print(file)
    text.pop()
    lang = []
    for i in text:
        l = detect(i)
        if l == 'ro' or l == 'ca':
            lang.append('co')
        elif l == 'it':
            test = i.split()
            count_u = test.count('u')
            count_il = test.count('il')
            if count_il > count_u:
                lang.append(l)
            else:
                lang.append('co')
        else:
            lang.append(l)
    
    
    aut = []
    for t in text:
        t = re.sub(r"’", r"'",t)
        dic = {}
        for i in list_aut_str:
            if i in t[-70:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            aut.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            aut.append('Inconnu')

    typ = []
    list_rub = ['art','let','poe','div','the','cha','feu','his','trd']
    for t in text:
        dic = {}
        for i in list_rub:
            if i in t[-5:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            typ.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            typ.append('Non défini')

    reg = []
    for r in text:
        dic = {}
        for i in list_regiolect:
            if i in r[-100:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            reg.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            reg.append('Inconnu')
            
    df = pd.DataFrame([text,lang,aut,typ,reg]).T
    df.columns = ['texte','langue','auteur','type','regiolecte']
    xml = df.to_xml()
    myre = '(\d{4}_\d+)'
    if re.search(myre, file):
        nfile = re.search(myre, file).group(1)
    with open(nfile + '.xml', "w+") as f:
        f.write(xml)

./text_section/1923_29.txt
./text_section/1923_32.txt
./text_section/1923_23.txt
./text_section/1923_39.txt
./text_section/1923_36.txt
./text_section/1923_38.txt
./text_section/1923_26.txt
./text_section/1923_22.txt
./text_section/1923_30.txt
./text_section/1923_37.txt
./text_section/1923_31.txt
./text_section/1923_24.txt
./text_section/1923_34.txt
./text_section/1923_28.txt
./text_section/1923_40.txt
./text_section/1923_43.txt
./text_section/1923_44.txt
./text_section/1923_35.txt
./text_section/1923_27.txt
./text_section/1923_33.txt
./text_section/1923_21.txt


### Modification des balises XML pour qu'elles correspondent à nos standards

In [6]:
list_xml_files = glob.glob("*.xml")
list_xml_files

['1923_40.xml',
 '1923_43.xml',
 '1923_30.xml',
 '1923_37.xml',
 '1923_21.xml',
 '1923_23.xml',
 '1923_36.xml',
 '1923_32.xml',
 '1923_29.xml',
 '1923_31.xml',
 '1923_35.xml',
 '1923_33.xml',
 '1923_24.xml',
 '1923_28.xml',
 '1923_44.xml',
 '1923_22.xml',
 '1923_39.xml',
 '1923_27.xml',
 '1923_38.xml',
 '1923_34.xml',
 '1923_26.xml']

In [7]:
for file in list_xml_files:
    xml_file = ET.parse(file)
    rootElement = xml_file.getroot()
    for element in rootElement.iter('data'):
        element.tag = 'revue'
    for element in rootElement.iter('row'):
        element.tag = 'rubrique'
    #item = rootElement.find("rubrique")
    #for element in item.iter('rubrique'):
        #element.tag = 'manchette'
    xml_file.write(file)


In [8]:
for file in list_xml_files: 
    with open(file, 'r') as f:
        content = f.read()
    soup = bs.BeautifulSoup(content,'xml')
    for tag in soup.find_all("langue"):
        tag.parent['lan'] = tag.get_text()
    for tag in soup.find_all("type"):
        tag.parent['typ'] = tag.get_text()
    for tag in soup.find_all("regiolecte"):
        cont = tag.get_text()
        if re.search("Parlata di a Custera",cont):
            tag.parent['regio'] = "custera"
        elif re.search("Parlata balanina",cont):
            tag.parent['regio'] = "balagna"
        elif re.search("Parlatu di a Castagniccia",cont):
            tag.parent['regio'] = "castagniccia"
        elif re.search("Parlata di Sartè",cont):
            tag.parent['regio'] = "sarte"
        elif re.search("Parlata di Levie",cont):
            tag.parent['regio'] = "levie"
        elif re.search("Parlata di Petretu",cont):
            tag.parent['regio'] = "petretu"
        elif re.search("Dialettu aiaccinu",cont):
            tag.parent['regio'] = "aiaccinu"
        else:
            tag.parent['regio'] = "inc"
    with open(file, 'w') as f:
        f.write(soup.prettify())

In [9]:
list_rub = ['art','let','poe','div','the','cha','feu','his','trd']
for file in list_xml_files:
    with open(file, 'r') as f:
        content = f.read()
    soup = bs.BeautifulSoup(content,'xml')
    for tag in soup.find_all("texte"):
        t = tag.get_text()
        for i in list_rub:
            if i in t[-7:]:
                t = re.sub(" " + i + r'\n','', t)
        for i in list_aut_str:
            if i in t[-30:]:
                t = re.sub(" " + i,'', t)            
        tag.string.replace_with(t)
    with open(file, 'w') as f:
        f.write(soup.prettify())

-----

### Vérifions que les ficheirs XML traités sont valides avant de les déplacer

In [10]:
def parsing_xml_corsican_press(file):
    '''
    Fonction permettant de parser des XML (avec DTD spécifique), pour retourner des listes de chaque élément.
    
    Entrée : XML
    Sortie : listes
    '''

    types = []
    langue = []
    texte = []
    auteur = []
    fichier = []
    regiolecte = []

    tree = etree.parse(file)
    
    for typ in tree.xpath("//type/text()"):
        typ = re.sub(r"\n ", "", typ)
        typ = re.sub(r"\n", "", typ)
        typ = re.sub(r"  ", "", typ)
        types.append(typ)
        fichier.append(file)

    for lan in tree.xpath("//langue/text()"):
        lan = re.sub(r"\n ", "", lan)
        lan = re.sub(r"\n", "", lan)
        lan = re.sub(r"  ", "", lan)
        langue.append(lan)

    for tex in tree.xpath("//texte/text()"):
        tex = re.sub(r"\n ", "", tex)
        tex = re.sub(r"\n", "", tex)
        texte.append(tex)

    for aut in tree.xpath("//auteur/text()"):
        aut = re.sub(r"\n ", "", aut)
        aut = re.sub(r"\n", "", aut)
        aut = re.sub(r"  ", "", aut)
        auteur.append(aut)

    for reg in tree.xpath("//regiolecte/text()"):
        reg = re.sub(r"\n ", "", reg)
        reg = re.sub(r"\n", "", reg)
        reg = re.sub(r"  ", "", reg)
        regiolecte.append(reg)
    
    
    
    return auteur, types, langue, texte, regiolecte, fichier

In [11]:
all_author = []
all_types = []
all_lang = []
all_text = []
all_regio = []
all_file = []

for file in list_xml_files:
    
    print(file)
    author, typo, lang, text, regiolecte, fichier = parsing_xml_corsican_press(file)
    all_author = all_author + author
    all_types = all_types + typo
    all_lang = all_lang + lang
    all_text = all_text + text
    all_regio = all_regio + regiolecte
    all_file = all_file + fichier

1923_40.xml
1923_43.xml
1923_30.xml
1923_37.xml
1923_21.xml
1923_23.xml
1923_36.xml
1923_32.xml
1923_29.xml
1923_31.xml
1923_35.xml
1923_33.xml
1923_24.xml
1923_28.xml
1923_44.xml
1923_22.xml
1923_39.xml
1923_27.xml
1923_38.xml
1923_34.xml
1923_26.xml


In [12]:
len(all_regio)

37

In [13]:
data = {"Auteur":all_author, "Type":all_types, "Langue":all_lang, "Régiolecte":all_regio, "Texte":all_text, "Fichier":all_file}

#data['Titre'].append(titre)
#data['Auteur'].append(auteur)
#data['Type'].append(types)
#data['Langue'].append(langue)
#data['Texte'].append(texte)

df_data = pd.DataFrame.from_dict(data)
df_data.to_csv("test_donnees.csv")

In [14]:
df_data

,Auteur,Type,Langue,Régiolecte,Texte,Fichier
0,Cervoni,art,fr,Inconnu,Mal doué pour la « Critique » et surtout à c...,1923_40.xml
1,Fernand Lombardi,art,fr,Inconnu,Une Initiative Episcopale C'est avec ces pa...,1923_40.xml
2,Fernand Lombardi,art,fr,Inconnu,"Après Ambroise, dont nous dirons un mot tout...",1923_43.xml
3,Cervoni,art,fr,Inconnu,"Le domaine Ùdu cep s'étend, empiétant sur ce...",1923_30.xml
4,Martinu Appinzapalu,his,co,Inconnu,Zia Paghjulara colla in Paradisu U paese er...,1923_30.xml
5,Fernand Lombardi,art,fr,Inconnu,Pour ceux qui dorment Il y a quelque chose ...,1923_37.xml
6,G. P. Lucciardi,art,co,Inconnu,A lingua côrsa Ognunu duvarebbe sentesi mur...,1923_37.xml
7,Pierre Rocca,art,fr,Inconnu,Notre Action Anti-francaise Trente et un lu...,1923_21.xml
8,Cervoni,art,fr,Inconnu,La dernière mêlée électorale nous a laissés ...,1923_23.xml
9,Martinu Appinzapalu,his,co,Inconnu,"Ai dodecianni, Inton-Francescu, un volse più...",1923_23.xml


### Classons les fichiers

In [15]:
#for file in glob.glob("*.xml"):
#    shutil.move(file, parent + "/xml/")